<a href="https://colab.research.google.com/github/kurosakiichig/sw-mid/blob/main/claw_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Claw‑Machine Manager ·  Demo (DeepSeek API)


In [1]:
!pip -q install fastapi uvicorn pyngrok sqlmodel nest_asyncio requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00


In [2]:

# === 密钥设置 ===
import os
from pyngrok import ngrok
NGROK_AUTHTOKEN = "2wFDwOVo0v0rYngb7CBWfVqyKjX_6XkBmGWKwxFhmVz59E2fo"
DEEPSEEK_API_KEY = "sk-b44c2b6a72684c718c0851bbf6dbf146"

ngrok.set_auth_token(NGROK_AUTHTOKEN)


In [3]:

from sqlmodel import SQLModel, Field, create_engine, Session, select
import nest_asyncio, threading, subprocess, time, json, requests
nest_asyncio.apply()
SQLModel.metadata.clear()
engine=create_engine("sqlite:///claw.db")
class Item(SQLModel, table=True, extend_existing=True):
    id:int|None=Field(default=None, primary_key=True)
    machine_id:str
    item_name:str
    stock:int
    wins:int=0
SQLModel.metadata.create_all(engine)
seed=[{"machine_id":"M02","item_name":"皮卡丘","stock":10,"wins":3}]
with Session(engine) as s:
    if not s.exec(select(Item)).first():
        s.add_all([Item(**r) for r in seed]); s.commit()
print("DB init done")


DB init done


In [4]:

from fastapi import FastAPI, HTTPException
app=FastAPI()
@app.get("/status/{mid}")
def status(mid:str):
    with Session(engine) as sess:
        rows=sess.exec(select(Item).where(Item.machine_id==mid)).all()
        if not rows: raise HTTPException(404)
        return rows
def run_uvicorn():
    subprocess.Popen(["uvicorn","app:app","--host","0.0.0.0","--port","8000"])
threading.Thread(target=run_uvicorn,daemon=True).start()
time.sleep(3)


In [5]:

# ngrok 单隧道
for t in ngrok.get_tunnels():
    ngrok.disconnect(t.public_url)
public_url=ngrok.connect(8000).public_url
print("Public URL:", public_url)


Public URL: https://4d14-35-238-69-191.ngrok-free.app


In [11]:

def ds_chat(prompt:str):
    url="https://api.deepseek.com/v1/chat/completions"
    headers={"Authorization":f"Bearer {DEEPSEEK_API_KEY}","Content-Type":"application/json"}
    payload={"model":"deepseek-chat","messages":[{"role":"user","content":prompt}]}
    r=requests.post(url,headers=headers,json=payload,timeout=60)
    r.raise_for_status()
    return r.json()["choices"][0]["message"]["content"]

def smart_chat(q:str):
    if "how many" in q:
        data=requests.get(f"{public_url}/status/M02").json()
        return f"Inventory details: {data}"
    return ds_chat(q)

print(smart_chat("How many Pikachu are left in Doll Machine No. 2?"))


The number of Pikachu left in Doll Machine No. 2 depends on the specific arcade or location you're referring to, as these machines are restocked manually by staff. There's no universal or real-time tracking system for such information.  

To find out:  
1. **Ask the arcade staff** – They can check or restock the machine if needed.  
2. **Inspect the machine visually** – Some machines have transparent cases where you can count remaining prizes.  
3. **Check if the arcade has a digital tracker** – Rarely, some modern machines display remaining stock.  

If you're referring to a *specific game or fictional scenario* (e.g., Pokémon lore), let me know! Otherwise, your best bet is to visit the location or call ahead.  

Would you like tips on winning Pikachu from the machine? 😊
